#### Passing DRS ids to the right DRS server

This notebook covers two related topics about DRS
* Sending DRS requests to the right server 
* Two different kinds of DRS URI

Any regular DRS server only responds to the local ids known to it.

For example, the BioDataCatalyst DRS server knows about and can resolve the following local id
* e747c529-a6ee-415f-90b8-e2db631f8ed9

To make that globally unique the prefix dg.4503 was registered for BioDataCatalyst. This allows the id to be referred to globally as 
* dg.4503:e747c529-a6ee-415f-90b8-e2db631f8ed9

### General metaresolvers
* http://identifiers.org/dg.4503:e747c529-a6ee-415f-90b8-e2db631f8ed9
* http://bioregistry.io/dg.4503:e747c529-a6ee-415f-90b8-e2db631f8ed9

These general metaresolvers are capable of sending many types of identifiers

icdc:000009
ncit:C15492
dbgap:phs000200.v11.p3


### A metaresolver for DRS
We need a tool that knows about different DRS servers and how to pass our request to the right one.
Such a tool is but will take DRS ids and send them to the DRS Server which can resolve them.


The MetaResolver will:
* Resolve compact ids
* Resolve host based DRS ids
* Send DRS calls to the correct server
* Handle id's prefixed with drs:// or not

 
The DRS Metaresolver appears like a DRS Client. It supports the two basic DRS functions
* get_object
* get_access_url

 🖐 Run the following examples and review how each kind of DRS id is handled


#### Step 1: As in previous examples create a client
This time our client is a metaresolver

In [ ]:
from fasp.loc import DRSMetaResolver
drs_mr = DRSMetaResolver(debug=False)

#### Step 2: Resolve a prefixed DRS id


In [ ]:
drs_mr.get_object('dg.4503:e747c529-a6ee-415f-90b8-e2db631f8ed9')

#### Step 3: Define a function to use the Metaresolver to send the DRS URI to the right server

As we are going to work with multiple lists of DRS ids we'll define a function to send a list of ids to the Metaresolver.

Click on the following cell to define the function.

 🖐 Expert option: In the following function you may edit what is printed for each DRS response. Use the example above to see what attributes may be listed.

In [ ]:
def check_list(id_list):
    for drs_id in id_list:
        print(f"DRS URI {drs_id}")
        drs_response = drs_mr.get_object(drs_id)
        num_of_methods = len(drs_response['access_methods'])
        print(f"Full response\n{drs_response}")
        print(f"has {num_of_methods} access_methods")
        print('_'*80)

#### Step 4: Resolve compact (prefix based ids)
The following is a list of DRS URIs with different prefixes

In [ ]:
prefixedIDs = [
            'dg.4503:66eeec21-aad0-4a77-8de5-621f05e2d301',
            'dg.4DFC:0e3c5237-6933-4d30-83f8-6ab721096bc7',
            'dg.ANV0:895c5a81-b985-4559-bc8e-cecece550756',
            #'dg.F82A1A:e6eecb29-1ae4-4f65-ae83-9ecf1c496de1',
            #'dg.MD1R:f55b8fed-a938-4cd7-8f39-5ee3cb75c218',
]

check_list(prefixedIDs)

#### Step 5: Resolve host based ids

In [ ]:
host_based_IDs = [
    'drs://gen3.biodatacatalyst.nhlbi.nih.gov/66eeec21-aad0-4a77-8de5-621f05e2d301',
    'drs://nci-crdc.datacommons.io/0e3c5237-6933-4d30-83f8-6ab721096bc7',
    'drs://gen3.theanvil.io/895c5a81-b985-4559-bc8e-cecece550756',
    'drs://data.kidsfirstdrc.org/e6eecb29-1ae4-4f65-ae83-9ecf1c496de1',
    'drs://data.midrc.org/f55b8fed-a938-4cd7-8f39-5ee3cb75c218',
    'drs://locate.be-md.ncbi.nlm.nih.gov/81b75c18e5def027579f9441f987b8a8'
]
check_list(host_based_IDs)

#### Step 6: Mixed list
* The following shows that lists of mixed host based and CURIEs can be handled
* The DRS Metaresolver can handle DRS ids with or without the drs:// (scheme) on the front

In [ ]:
mixedIDs = [
            
    # Curie with scheme
    'drs://dg.4503:66eeec21-aad0-4a77-8de5-621f05e2d301',
    # Curie no scheme
    'dg.4DFC:0e3c5237-6933-4d30-83f8-6ab721096bc7',
    # Host based with scheme
    'drs://dg.ANV0:895c5a81-b985-4559-bc8e-cecece550756',
    # Host based no scheme
    'locate.be-md.ncbi.nlm.nih.gov/81b75c18e5def027579f9441f987b8a8'
]

check_list(mixedIDs)